In [4]:
import cv2
import numpy as np
import math
import copy

img = cv2.imread("14531751641yu3bw.jpg")
#cv2.imshow("org", img)

#cv2.imwrite("result.jpg",img);

print(img.shape)

color = np.array([[0 for color in range(3)] for i in range(256)])

print(color.shape)

Negative = img#np.zeros((img.shape[0],img.shape[0],3), 'uint8')

for i in range(0,Negative.shape[0]):
    for j in range(0,Negative.shape[1]):
        Negative[i,j,0] = 255 - img[i,j,0]
        color[Negative[i,j,0]][0] +=1
        Negative[i,j,1] = 255 - img[i,j,1]
        color[Negative[i,j,1]][1] +=1
        Negative[i,j,2] = 255 - img[i,j,2]
        color[Negative[i,j,2]][2] +=1

#for i in range(color.shape[0]):
#    print(i,color[i],"\n")
#0 [  0   3 909] 
#1 [  1   2 243] 
#2 [  0   4 324] 
#   ...
#253 [1589  863 2108] 
#254 [ 964  674 1006] 
#255 [3561 1341 3524] 
valmax1 = 0
valmax2 = 0
valmax3 = 0
for i in range(color.shape[0]):
    if valmax1 < color[i][0]:
        valmax1 = color[i][0]
    if valmax2 < color[i][1]:
        valmax2 = color[i][1]
    if valmax3 < color[i][2]:
        valmax3 = color[i][2]

#cv2.imshow("Negative", Negative)



log = img #np.zeros((img.shape[0],img.shape[0],3), 'uint8')

c = 1.5
for i in range(0,log.shape[0]):
    for j in range(0,log.shape[1]):
        if (c*math.log(1+img[i,j,0],2)) < 0:
            log[i,j,0] = 0
        else:
            log[i,j,0] = np.int(c*math.log(1+img[i,j,0],2))
        if (c*math.log(1+img[i,j,1],2)) < 0:
             log[i,j,1] = 0
        else:
            log[i,j,1] = np.int(c*math.log(1+img[i,j,1],2))
        if (c*math.log(1+img[i,j,2],2)) < 0:
             log[i,j,2] = 0
        else:
            log[i,j,2] = np.int(c*math.log(1+img[i,j,2],2))
        
#cv2.imshow("log", log)
        
    
gamma = img
for i in range(0,log.shape[0]):
    for j in range(0,log.shape[1]):
        gamma[i,j,0] = np.int(math.gamma(1+img[i,j,0]))
        gamma[i,j,1] = np.int(math.gamma(1+img[i,j,1]))
        gamma[i,j,2] = np.int(math.gamma(1+img[i,j,2]))
        
#cv2.imshow("gamma", gamma)

#min[3][3] = {1,...}
#for i in range(0,log.shape[0]):
#    for j in range(0,log.shape[1]):
#        for k in range(-1,1):
#            for l in range(-1,1):
    

def image_histogram_equalization(image):
    hist, bins = np.histogram(image.flatten(), 256,[0,256])

    cdf = hist.cumsum()

    # cdf의 값이 0인 경우는 mask처리를 하여 계산에서 제외
    # mask처리가 되면 Numpy 계산에서 제외가 됨
    # 아래는 cdf array에서 값이 0인 부분을 mask처리함
    cdf_m = np.ma.masked_equal(cdf,0)
    
    #History Equalization 공식
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    
    # Mask처리를 했던 부분을 다시 0으로 변환
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    return cdf[image]

#img = cv2.imread("chapter3/planet_glow.jpg")
cv2.imshow("org", img)

hist_b = image_histogram_equalization(img[:,:,0])
hist_g = image_histogram_equalization(img[:,:,1])
hist_r = image_histogram_equalization(img[:,:,2])

#cv2.imshow("histogram_equalization_b",hist_b)
#cv2.imshow("histogram_equalization_g",hist_g)
#cv2.imshow("histogram_equalization_r",hist_r)

rgbArray = np.dstack((hist_b,hist_g,hist_r))
cv2.imshow("rgbArray",rgbArray)

cv2.waitKey()
cv2.destroyAllWindows()


(796, 640, 3)
(256, 3)


In [8]:
import cv2
import numpy as np
img = cv2.imread("14531751641yu3bw.jpg")

kernal = np.array([[1/9,1/9,1/9],[1/9,1/9,0/9],[1/9,1/9,1/9]])

average = np.zeros((img.shape[0],img.shape[1],3), 'uint8')

print(average.shape)

cv2.imshow("org",img)

for i in range(1,average.shape[0]-1):
    for j in range(1,average.shape[1]-1):
        for k in range(0,2):
            for l in range(0,2):
                average[i,j,0] += int(img[i,j,0] * kernal[k][l])
                average[i,j,1] += int(img[i,j,1] * kernal[k][l])
                average[i,j,2] += int(img[i,j,2] * kernal[k][l])

cv2.imshow("Convolution_average",average)

cv2.waitKey()
cv2.destroyAllWindows()

(448, 663, 3)


In [22]:
kernal_laplacian = np.array([[1,1,1],[1,-8,1],[1,1,1]])
kernal_laplacian2 = np.array([[0,1,0],[1,-4,1],[0,1,0]])
laplacian  = np.zeros((img.shape[0],img.shape[1],3), 'uint8')

print(average.shape)


for i in range(1,laplacian.shape[0]-1):
    for j in range(1,laplacian.shape[1]-1):
        for k in range(0,2):
            for l in range(0,2):
                laplacian[i,j,0] += int(img[i+k-1,j+l-1,0] * kernal_laplacian[k][l])
                laplacian[i,j,1] += int(img[i+k-1,j+l-1,1] * kernal_laplacian[k][l])
                laplacian[i,j,2] += int(img[i+k-1,j+l-1,2] * kernal_laplacian[k][l])

cv2.imshow("Convolution_laplacian",laplacian)

cv2.waitKey()
cv2.destroyAllWindows()

(448, 663, 3)


In [37]:
def bubble_sort(random_list):    
    for start_index in range( len(random_list)-1 ):
        for index in range( 1, len(random_list) - start_index ):
            if random_list[index-1] > random_list[index]:
                temp = random_list[index-1]
                random_list[index-1] = random_list[index]
                random_list[index] = temp
                
    return random_list[int(len(random_list)/2)]

median  = np.zeros((img.shape[0],img.shape[1],3), 'uint8')

median_array = np.zeros((3,9), 'uint8')

for i in range(1,median.shape[0]-1):
    for j in range(1,median.shape[1]-1):
        cnt = 0
        for k in range(0,3):
            for l in range(0,3):
                median_array[0,cnt]=img[i+k-1,j+l-1,0]
                median_array[1,cnt]=img[i+k-1,j+l-1,1]
                median_array[2,cnt]=img[i+k-1,j+l-1,2]
                cnt = cnt+1
                      
        median[i,j,0] = bubble_sort(median_array[0])
        median[i,j,1] = bubble_sort(median_array[1])
        median[i,j,2] = bubble_sort(median_array[2])

cv2.imshow("Convolution_median",median)

cv2.waitKey()
cv2.destroyAllWindows()

In [38]:
def bubble_sort_Max(random_list):    
    for start_index in range( len(random_list)-1 ):
        for index in range( 1, len(random_list) - start_index ):
            if random_list[index-1] > random_list[index]:
                temp = random_list[index-1]
                random_list[index-1] = random_list[index]
                random_list[index] = temp
                
    return random_list[len(random_list)-2]

Max_img  = np.zeros((img.shape[0],img.shape[1],3), 'uint8')

Max_array = np.zeros((3,9), 'uint8')

cv2.imshow("org",img)
cnt = 0
for i in range(1,Max.shape[0]-1):
    for j in range(1,Max.shape[1]-1):
        cnt = 0
        for k in range(0,3):
            for l in range(0,3):
                Max_array[0,cnt]=img[i+k-1,j+l-1,0]
                Max_array[1,cnt]=img[i+k-1,j+l-1,1]
                Max_array[2,cnt]=img[i+k-1,j+l-1,2]
                cnt = cnt+1
                      
        Max_img[i,j,0] = bubble_sort_Max(Max_array[0])
        Max_img[i,j,1] = bubble_sort_Max(Max_array[1])
        Max_img[i,j,2] = bubble_sort_Max(Max_array[2])

cv2.imshow("Convolution_Max",Max_img)

cv2.waitKey() 
cv2.destroyAllWindows()

9


In [35]:
def bubble_sort_Min(random_list):    
    for start_index in range( len(random_list)-1 ):
        for index in range( 1, len(random_list) - start_index ):
            if random_list[index-1] > random_list[index]:
                temp = random_list[index-1]
                random_list[index-1] = random_list[index]
                random_list[index] = temp
                
    return random_list[0]

Min  = np.zeros((img.shape[0],img.shape[1],3), 'uint8')

Min_array = np.zeros((3,9), 'uint8')

for i in range(1,Min.shape[0]-1):
    for j in range(1,Min.shape[1]-1):
        cnt = 0
        for k in range(0,3):
            for l in range(0,3):
                Min_array[0,cnt]=img[i+k-1,j+l-1,0]
                Min_array[1,cnt]=img[i+k-1,j+l-1,1]
                Min_array[2,cnt]=img[i+k-1,j+l-1,2]
                cnt = cnt+1
                      
        Min[i,j,0] = bubble_sort_Min(Min_array[0])
        Min[i,j,1] = bubble_sort_Min(Min_array[1])
        Min[i,j,2] = bubble_sort_Min(Min_array[2])

cv2.imshow("Convolution_Min",Min)

cv2.waitKey()
cv2.destroyAllWindows()